In [1]:
import re
import tqdm
import pandas as pd
subs=pd.read_csv('./COVID19positive_submissions.csv', low_memory=False)
coms=pd.read_csv('./COVID19positive_comments.csv', low_memory=False)

In [2]:
def fn(df):
  if type(df.selftext)==float:
    return df.title
  else:
    return df.title + ' ' + df.selftext

In [3]:
subs['text']=subs.apply(fn, axis=1)
coms=coms[coms.body.notna()]

In [7]:
f=['father', 'dad', 'mother', 'mom', 'brother', 'sister', 'family', 'aunt', 'uncle', 'grand']
s=['fever', 'cough', 'nausea', 'vomit', 'headache', 'sore', 'sense', 'smell', 'nose']
g=['thank', 'lov', 'pray']
r=['recover', 'hop', 'faith', 'rest', 'sleep']

pt_f= '\w*|'.join(f) + '\w+'
pt_s= '\w*|'.join(s) + '\w*'
pt_g= '\w*|'.join(g) + '\w*'
pt_r= '\w*|'.join(r) + '\w*'

posts_family_symptom=[]
comments_gratitude_recovery=[]

for i, sub in tqdm.tqdm(subs.iterrows(), total=subs.shape[0]):
  d={}
  m_f = re.findall(pt_f, sub.text, flags=re.IGNORECASE)
  m_s = re.findall(pt_s, sub.text, flags=re.IGNORECASE)
  d['id']=sub.id
  if len(m_f)!=0 & len(m_s)==0:
    d['family']=1
  if len(m_s)!=0 & len(m_f)==0:
    d['symptom']=1
  posts_family_symptom.append(d)

for i, com in tqdm.tqdm(coms.iterrows(), total=coms.shape[0]):
  d={}
  m_g = re.findall(pt_g, com.body, flags=re.IGNORECASE)
  m_r = re.findall(pt_r, com.body, flags=re.IGNORECASE)
  d['id']=com.id
  if len(m_g)!=0 & len(m_r)==0:
    d['gratitude']=1
  if len(m_r)!=0 & len(m_g)==0:
    d['recovery']=1
  comments_gratitude_recovery.append(d)

100%|██████████| 993014/993014 [02:01<00:00, 8147.70it/s]


In [8]:
df_f_s=pd.DataFrame(posts_family_symptom)
df_g_r=pd.DataFrame(comments_gratitude_recovery)
print(df_f_s.shape[0], df_g_r.shape[0])
df_f_s.id.nunique(), df_g_r.id.nunique()

93576 993014


(93576, 993014)

In [10]:
df_f_s=df_f_s.fillna(0)
df_g_r=df_g_r.fillna(0)

,id,family,symptom
0,fiag4h,0.0,0.0
1,fid080,0.0,0.0
2,fj4l50,0.0,0.0
3,fj963y,0.0,0.0
4,fjcfsv,0.0,0.0
...,...,...,...
93571,100628x,0.0,1.0
93572,1006i9z,1.0,1.0
93573,1006kys,0.0,0.0
93574,1006n2k,0.0,0.0


In [11]:
df_f_s.to_csv('posts_family_symptom.csv', index=False)
df_g_r.to_csv('comments_gratitude_recovery.csv', index=False)